In [2]:
import sys
import pathlib
import pandas as pd
import numpy as np
import boto3

ROOT = pathlib.Path().absolute().parent.as_posix()
if ROOT not in sys.path:
    sys.path.append(ROOT)

In [3]:
print(sys.path)

['/root/AWS ASDI Competition/ASDI-Hackathon/Scripts', '/opt/conda/lib/python37.zip', '/opt/conda/lib/python3.7', '/opt/conda/lib/python3.7/lib-dynload', '', '/opt/conda/lib/python3.7/site-packages', '/opt/conda/lib/python3.7/site-packages/IPython/extensions', '/root/.ipython', '/root/AWS ASDI Competition/ASDI-Hackathon']


In [5]:
!pip install sagemaker-studio-image-build

  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker-studio-image-build: filename=sagemaker_studio_image_build-0.6.0-py3-none-any.whl size=13469 sha256=0ab1ac230de3d54b06c27723a66de582ffb9a66f9196449332a005b0d51143bf
  Stored in directory: /root/.cache/pip/wheels/c1/9c/e8/cbf0266d9d9b1b6161f7ba9ddf572d02aacd411e8a5b4d186b
Successfully built sagemaker-studio-image-build

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [6]:
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name
ecr_repository = 'asdi-hackathon-sagemaker-processing-container'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

# Create ECR repository and push docker image
!docker build -t $ecr_repository docker
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $processing_repository_uri
!docker push $processing_repository_uri

/bin/bash: docker: command not found
/bin/bash: docker: command not found
Exception ignored in: <_io.TextIOWrapper name='<stdout>' mode='w' encoding='UTF-8'>
BrokenPipeError: [Errno 32] Broken pipe
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:eu-west-2:991775080983:repository/asdi-hackathon-sagemaker-processing-container",
        "registryId": "991775080983",
        "repositoryName": "asdi-hackathon-sagemaker-processing-container",
        "repositoryUri": "991775080983.dkr.ecr.eu-west-2.amazonaws.com/asdi-hackathon-sagemaker-processing-container",
        "createdAt": 1661706197.0,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}
/bin/bash: docker: command not found
/bin/bash: docker: command not found


In [4]:
from sagemaker import get_execution_role
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

#role = 'role_name_with_sagemaker_permissions' #i.e. my sagemaker role otherwise proceed inside sagemaker and will auto find
role = get_execution_role()

script_processor = ScriptProcessor(command=['python3'], 
                            image_uri=processing_repository_uri, role=role, 
                            instance_type="ml.t3.large", instance_count=1)

NameError: name 'processing_repository_uri' is not defined

In [11]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

script_processor.run(
    code='processing.py',
    #no inputs technically needed as script is self-sufficient in this regard
    #inputs=[ProcessingInput(source=input_data, destination="asdi-hackathon/*")],
    outputs=[ProcessingOutput(output_name="final_df", source="asdi-hackathon/final-data/final_df.csv")]
)

processing_job_description = script_processor.jobs[-1].describe()
print(processing_job_description)

ValueError: code Scripts/processing.py wasn't found. Please make sure that the file exists.
                    